In this notebook we will learn how to create and train an embedding layer for the words appearing in a text data. We will then train a simple DNN based model to do sentiment analysis on this data. 

## Exercise

This is exercise 13.10 in [this](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) book.

### Problem Statement

In this exercise you will download a dataset, split it, create a tf.data.Dataset to load it and preprocess it efficiently, then build and train a binary classification model containing an Embedding layer:

  - a. Download the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), which contains 50,000 movies reviews from the Internet Movie Database. The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise.
  
    
  - b. Split the test set into a validation set (15,000) and a test set (10,000).
  
  
  - c. Use tf.data to create an efficient dataset for each set.
  
  
  - d. Create a binary classification model, using a TextVectorization layer to preprocess each review. If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method.
  
  
  - e. Add an Embedding layer and compute the mean embedding for each review, multiplied by the square root of the number of words (see Chapter 16). This rescaled mean embedding can then be passed to the rest of your model.
  
  
  - f. Train the model and see what accuracy you get. Try to optimize your pipelines to make training as fast as possible.


  - g. Use TFDS to load the same dataset more easily: tfds.load("imdb_reviews").

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print('tensorflow version: {}'.format(tf.__version__))
print('keras version: {}'.format(keras.__version__))

tensorflow version: 2.1.0
keras version: 2.2.4-tf


In [2]:
import os
print('cwd: {}'.format(os.getcwd()))

cwd: /home/prarit/MachineLearningProjects/Word-Embeddings


### Downloading the Large Movie Review Dataset

In [3]:
# good tutorial on using wget: https://www.tecmint.com/download-and-extract-tar-files-with-one-command/
# turn off verbose output of wget using the flag -nv : https://shapeshed.com/unix-wget/#how-to-turn-off-verbose-output 
!wget -c -nv http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -o - 

In [4]:
# uncompress the downloaded files
!tar xzf aclImdb_v1.tar.gz

Note that tensorflow also provides this dataset: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb

### Briefly Explore the Dataset

In [5]:
# list the files in the current working directory
os.listdir()

['README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'aclImdb',
 'Word-Embeddings.ipynb',
 'aclImdb_v1.tar.gz']

We see that aclImdb_v1.tar.gz was extracted to a folder called aclImdb. Let's see the contents of this file.

In [6]:
path = os.path.join(os.getcwd() , 'aclImdb')
contents = os.listdir(path)
print('The contents of aclImdb are: \n{}'.format(contents))

The contents of aclImdb are: 
['imdb.vocab', 'train', 'README', 'imdbEr.txt', 'test']


There is a README file in aclImdb, let us read it.

In [7]:
# read README
filepath = os.path.join(path, 'README')
with open(filepath, 'r') as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

From README, we find that the train/test folder contains a 'pos' folder for positive reviews and a 'neg' for negative reviews along with .txt files containing urls of positive and negative reviews respectively. There are also some other files for bag-of-words features etc. 

Each train/test folder contains a total of 25000 reviews of which 12500 are positive reviews and 12500 are negative reviews.

Let's verify the above about the 'train' folder

In [8]:
train_path = os.path.join(path,'train')
print('contents of the train folder: \n{}'.format(os.listdir(train_path)))

contents of the train folder: 
['unsup', 'urls_pos.txt', 'pos', 'labeledBow.feat', 'unsupBow.feat', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


In order to create a dataset, we need the path to all the reviews. We can create the corresponding list of paths using glob

In [9]:
import glob

In [10]:
# paths to the positive reviews in the training set
train_pos_path = os.path.join(train_path, 'pos', '*.txt')
train_pos_reviews = glob.glob(train_pos_path)
print('No. of train-set files with positive reviews: {}'.format(len(train_pos_reviews)))

No. of train-set files with positive reviews: 12500


In [11]:
# paths to the negative reviews in the training set
train_neg_path = os.path.join(train_path, 'neg', '*.txt')
train_neg_reviews = glob.glob(train_neg_path)
print('No. of train-set files with negative reviews: {}'.format(len(train_neg_reviews)))

No. of train-set files with negative reviews: 12500


let us give a brief look at a positive review. 

In [12]:
file = train_pos_reviews[0]
with open(file, 'r') as f:
    print(f.read())

I watch them all.<br /><br />It's not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it's completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don't watch. 'How she move.' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It's just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I'm a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn't all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more clearly set there. I 

Later, we will like to load an preprocess all the data using tensorflow's data API, therefore let us quickly see how to read the same file as above but this time by using tensorflow's [TextLineDataset](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) method.

In [13]:
pos_fl0 = tf.data.TextLineDataset(file)
for item in pos_fl0:
    print(item)

tf.Tensor(b'I watch them all.<br /><br />It\'s not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it\'s completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don\'t watch. \'How she move.\' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It\'s just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I\'m a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn\'t all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more c

Perfect! As expected, we see that the pos_fl0 contains a single item and it's value output matches the output of the previous code cell. 

### Preprocessing the reviews

Having, learnt how to use tf.data.TextLineData() method, we can now starting preprocessing the data. In order to do this, we notice that the review contains punctuation marks and html line break tags etc. We will have to write a preprocessing function to get rid of these. Additionally, we will also change all alphabets to lower case.

#### Removing line-brk tags

This can be very simply done by using the .replace() method of python strings. We can therefore use it to replace all occurrances of the line-break tag with a space. In tensorflow, the equivalent method is [tf.strings.regex_replace()](https://www.tensorflow.org/api_docs/python/tf/strings/regex_replace).

#### tf.strings.regex_replace
Note that 'regex' in regex_replace() stands for ["regular expression"](https://docs.python.org/3/howto/regex.html). For e.g. the following will work:

In [14]:
tf.strings.regex_replace('hello','e','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

But the following will throw an error: tf.strings.regex_replace('h(llo', '(','E')

This is because "(" is a metacharacter. To match and replace meta-characters, we must prepend a backslash before them. This can be done as follows: '\\\\' + char. 

The previous code cell can now be made to work

In [15]:
# The error thrown by this code cell is intentional
tf.strings.regex_replace('h(llo', '\\'+'(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [16]:
tf.strings.regex_replace('h(llo','\(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

##### Imp: 

Note that backslash itself is also a  meta-character. To search and replace backslash, we do the following:

In [17]:
tf.strings.regex_replace('h\llo', '\\'+'\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [18]:
# we use '\\\\' and NOT '\\' to search and replace a backslash
tf.strings.regex_replace('h\llo', '\\\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

We now try three different ways of removing punctuations from a tensorflow string and compare their timings:

1) punc_filter_and_to_lower1: use tf.strings.unicode_decode() to convert all the characters in the string into an array of their ascii codes. We then loop through this array, skipping over the places where the entry matches the ascii code of a punctuation. Finally we call tf.strings.unicode_encode() on this array to convert the ascii codes back to characters, thereby obtaining a string with all the punctuations stripped. 


2) punc_filter_and_to_lower2: use tf.regex_replace() to search and replace each punctuation by empty space. Pay special attention to prepend backslash in order to be able to use meta-characters in regex_replace.


3) punc_filter_and_to_lower3: simply extract the python string using its .numpy() method. Then simple iterate through the characters of the string, skipping over the punctutations. Join the resulting list of character using .join() method. 

4) punc_filter_and_to_lower4: Use [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) by setting it's "alphanum_only" arg to True. This way it will only parse alpha-numeric characters in the text and to split the text at occurrances of non-alphanumeric characters. Since whitespace is a non-alpha-numeric character, the output will largely consist of a list of words in the text.  The caeat with this approach is that words with apostrophe in them such as " don't " will be split into two words: "don" and "t". Also, [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) does NOT consider underscores as non-alpha-numeric, so underscores do not get removed form the text. On the other hand since, the tokenizer already generates a list of words, we use this list to generate a vocobulary of words in the dataset at this step it-self, making the preprocessing faster.

There timing on the first review in the training set was as follows:

1) punc_filter_and_to_lower1: Wall time: ~ 3 s

2) punc_filter_and_to_lower2: Wall time: ~ 8 ms

3) punc_filter_and_to_lower3: Wall time: ~ 6 ms

4) punc_filter_and_to_lower4: Wall time: ~ 6 ms (when not udating a vocabulary of words) 

5) 4) punc_filter_and_to_lower4: Wall time: ~ 7 ms (when also udating a vocabulary of words) 

Clearly, the last function is the fastest with the first one being extremely slow (takes several secs). 

In [19]:
import string

In [20]:
# list of punctuations
punc_ls = string.punctuation
print('punctuations before utf encoding: {}'.format(punc_ls))
punc_ls2 = tf.strings.unicode_decode(punc_ls, input_encoding = 'UTF-8')
print('punctuations after utf encoding: \n{}'.format(punc_ls2))

punctuations before utf encoding: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punctuations after utf encoding: 
[ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  58  59  60
  61  62  63  64  91  92  93  94  95  96 123 124 125 126]


In [21]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.unicode_encode(), tf.strings.unicode_decode()
def punc_filter_and_to_lower1(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = tf.strings.unicode_decode(st2, input_encoding = 'utf-8')
    st2 = tf.strings.unicode_encode([char for char in st2 if char not in punc_ls2], output_encoding = 'UTF-8')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [22]:
import time

In [23]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower1(item)

CPU times: user 3.51 s, sys: 42.3 ms, total: 3.55 s
Wall time: 3.55 s


In [24]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [25]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.regex_replace()
def punc_filter_and_to_lower2(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ')
    
    # now we replace all the punctions in the string
    for punc in punc_ls:
        # V.imp: to replace meta-characters we prepend a backslash to them 
        # infact we can also prepend a backslach before all the characters
        # to prepend a backslash before a character we do: '\\' + char
        st2 = tf.strings.regex_replace(st2, '\\'+ punc, ' ')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [26]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower2(item)

CPU times: user 8.1 ms, sys: 7 µs, total: 8.11 ms
Wall time: 7.73 ms


In [27]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all  it s not better than the amazing ones   strictly ballroom    shall we dance    japanese version   but it s completely respectable and pleasingly different in parts  i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting  for example  the  name should scream don t watch   how she move   since when can movie titles ignore grammar    there is nothing inherently incorrect about caribbean english grammar  it s just not canadian standard english grammar  comments about the dialogue seem off to me  i put on the subtitles because i m a canadian standard english speaker  so i just automatically assumed that i would have trouble understanding all of it  it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american  i loved that this movie was set in toronto and  in fact  wish it was even more clearly set there  i 

In [28]:
# function to get rid of punctuations, html line breaks and change to lower case
# using .join() method in python string class
def punc_filter_and_to_lower3(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = st2.numpy().decode('utf-8')
    st2 = ''.join([char for char in st2 if char not in punc_ls])
    
    st2 = st2.lower()
    
    st2 = tf.convert_to_tensor(st2)
    
    return st2

In [29]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower3(item)

CPU times: user 5.48 ms, sys: 253 µs, total: 5.73 ms
Wall time: 5.54 ms


In [30]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [31]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tfds.features.text.tokenizer()
# we will also simultaneously generate a vocabulary in this step
import tensorflow_datasets as tfds
def punc_filter_and_to_lower4(st, vocab = None):
    ''' vocab: vocabulary to update with the words in st'''
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.lower(tf.strings.regex_replace(st, line_brk_tag, ' '))
    
    # now we replace all the punctions in the string
    tokenizer = tfds.features.text.Tokenizer()
    words = tokenizer.tokenize(st2.numpy()) # note that the inpout has to be a python string NOT as tensor 
                                            # The output is a list NOT a tensor
                                            # https://stackoverflow.com/questions/56665868/tensor-numpy-not-working-in-tensorflow-data-dataset-throws-the-error-attribu
    st2 = tf.strings.join(words, ' ')
    
    if type(vocab) == set:
        vocab.update(words)
        return st2, vocab
    
    
    return st2

In [32]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower4(item)

CPU times: user 8.12 ms, sys: 8 µs, total: 8.12 ms
Wall time: 7.09 ms


In [33]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all it s not better than the amazing ones _strictly ballroom_ _shall we dance _ japanese version but it s completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream don t watch how she move since when can movie titles ignore grammar there is nothing inherently incorrect about caribbean english grammar it s just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because i m a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was

In [34]:
%%time
# test the timing of punc_filter_and_to_lower4 when simultaneously updating a dictionary
vocab2 = set([])
for item in pos_fl0:
    lst, vocab2 = punc_filter_and_to_lower4(item, vocab2)

CPU times: user 7.64 ms, sys: 181 µs, total: 7.82 ms
Wall time: 7.51 ms


### Build a Vocabulary based on the training data

This can be easily done using python's [set()](https://docs.python.org/3.8/library/stdtypes.html#set-types-set-frozenset) container.: basically, split each text in the training instance into words and update the set with this list. This will add any new words in the text to the set. In the end, the set will contain all the unique words in the training dataset.

Note that we have also implemented the above idea in our function punc_filter_and_to_lower4() 

In [35]:
# function to create a vocabulary using the set() container
vocab1 = set([]) # python set for containing unique words in the training dataset
def vocab_builder1(strng):
    # split the string into its words
    words = tf.strings.split(strng)
    # update vocab
    vocab1.update(words.numpy())
    return

In [36]:
%%time
for item in pos_fl0:
    item = punc_filter_and_to_lower3(item)
    vocab_builder1(item)

CPU times: user 258 ms, sys: 68.1 ms, total: 326 ms
Wall time: 324 ms


Note that the vocab_builder1() function as defined above takes about 300 ms. On the otherhand, in punc_filter_and_to_lower4() function,  we were able to get almost the same result (upto the caveats mentioned in the previous section) by using tfds.features.text.Tokenizer(), in about 8 ms. Clearly, there is a difference of few order of magnitude between the time taken by the two routines with the difference between their output being quite tolerable.  

#### building the vocabulary

In [37]:
vocab = set([])
train_filepaths = tf.data.Dataset.list_files([train_pos_path, train_neg_path])
train_dataset = tf.data.TextLineDataset(train_filepaths)

ctr = 0
for item in train_dataset:
    _ , vocab = punc_filter_and_to_lower4(item, vocab)
    
    ctr+=1
    if ctr%255 ==0:
        print("=", end = '')

In [38]:
# pick 5 random words from vocab to inspect it
import random

random.sample(vocab, 5)

['atomspheric', 'resumé', 'margarita', 'batfan', 'plagiary']

In [39]:
print('no. of words in vocabulary: {}'.format(len(vocab)))

no. of words in vocabulary: 74893


### Create a lookup table based on the vocabulary

In [40]:
indices = tf.range(len(vocab), dtype = tf.int64)
lookup_initializer = tf.lookup.KeyValueTensorInitializer( list(vocab), indices) 
num_oov_buckets = 50 
lookup_table = tf.lookup.StaticVocabularyTable( initializer = lookup_initializer, 
                                               num_oov_buckets = num_oov_buckets)

In [41]:
# function to map a text to integers using the above lookup table
def text_to_integers(X):
    X = tf.strings.split(X)
    int_ls = lookup_table.lookup(X)
    return int_ls

In [42]:
for item in pos_fl0:
    item = punc_filter_and_to_lower4(item)
    item = text_to_integers(item)
    print(item)

tf.Tensor(
[48439 30765 17638 41395 29831  9731 14234 12919 15503 21034 74794 42136
 17174 61205 55113  7643 12767 70810 52463 13481 29402 29831  9731 55955
 62738 10749 65857 60297 47311 52497 48439 18168 65369 20408 22965 10749
 48439 29469 61054  1918 21034 63179 29350 48361 47311 61054  1918  3753
 18842 27741 44434  1979 51064 21034 59712 11991 11652 54379 38416 30765
 64534 37251 19554 37041 38334 48207 25954 32409 60441    52  7845 21472
 62312 38233  7839 29350 23499 20408    52 29831  9731 54432 14234 67211
 13979 20408    52 16182 29350 21034 66763  2171 26295  6102 56828 48439
 12868  3537 21034 53374 54322 48439 42719 30010 67211 13979 20408 50287
 30805 48439 54432 25895 38108  3896 48439  8528 16643 68515 44947 41395
  1918 29831 29831 14903 38416 41395  3896 19169 10749 29831 42213 30010
 46024 60297 10128 62493 21034 34120 44748 34096 48439 16643 23472  6790
 30805 67059 48439 67071  3896 11817 25954 73286 26308 47311 13051 10749
 47311 57551 50118 29831 73286 59017 697

### Create an Embedding Matrix

In [43]:
embedding_dim = 100
initial_embeddings = tf.random.uniform([len(vocab) + num_oov_buckets, embedding_dim] ) 
embedding_matrix = tf.Variable(initial_embeddings)
embedding_matrix

<tf.Variable 'Variable:0' shape=(74943, 100) dtype=float32, numpy=
array([[0.27906322, 0.8319788 , 0.8671597 , ..., 0.59542024, 0.1292175 ,
        0.8779899 ],
       [0.27754378, 0.16927779, 0.9528674 , ..., 0.8340106 , 0.9525088 ,
        0.61001444],
       [0.1377747 , 0.51589966, 0.92480576, ..., 0.907174  , 0.9754183 ,
        0.71382284],
       ...,
       [0.31066298, 0.3120792 , 0.36700225, ..., 0.29035735, 0.48106587,
        0.5356971 ],
       [0.78949046, 0.0302062 , 0.29179502, ..., 0.9442512 , 0.07022405,
        0.5614449 ],
       [0.89836776, 0.06539917, 0.0664115 , ..., 0.640229  , 0.53680515,
        0.22554016]], dtype=float32)>

In [44]:
# function to create an embedding-vector from a list of word indices
def indices_to_embeddings(idxs):
    embeddings = tf.nn.embedding_lookup(embedding_matrix, idxs)
    sqrt_num_words = tf.math.sqrt(tf.cast(tf.size(idxs), tf.float32)) # size is of type int32 but 
                                                                      # tf.math.sqrt has to have one of 
                                                                      # bfloat16, half, float32, 
                                                                      # float64, complex64, 
                                                                      # complex128.
    mean_embedding = tf.math.reduce_mean(embeddings, axis = 0)
    sentence_embedding = tf.multiply(sqrt_num_words, mean_embedding)
    return sentence_embedding

In [45]:
for item in pos_fl0:
    item = punc_filter_and_to_lower4(item)
    item = text_to_integers(item)
    item = indices_to_embeddings(item)

In [46]:
item

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([8.946577 , 7.7086296, 7.889461 , 7.6378326, 7.7103095, 8.75428  ,
       7.635992 , 8.230398 , 7.0222974, 8.376617 , 7.9879985, 7.1177044,
       7.845598 , 8.805258 , 8.428925 , 7.9334426, 8.093625 , 7.972808 ,
       7.257497 , 8.336182 , 7.623115 , 7.361554 , 8.276004 , 6.9057245,
       8.051547 , 8.387872 , 8.48967  , 8.2096815, 8.696532 , 8.533066 ,
       7.271341 , 7.4471626, 8.351785 , 7.6975226, 8.140875 , 7.986603 ,
       7.381612 , 7.6213307, 8.292445 , 7.390688 , 8.303003 , 7.6313944,
       6.9432006, 7.4013   , 7.9718246, 8.071688 , 7.034995 , 8.841431 ,
       8.613527 , 7.964976 , 7.319157 , 6.8373456, 7.3675656, 8.248731 ,
       7.495907 , 7.3829293, 8.225588 , 8.1930895, 7.656006 , 7.452039 ,
       7.747517 , 6.536757 , 8.203257 , 8.861534 , 8.209468 , 8.0763645,
       7.8543854, 7.6970987, 8.034244 , 8.323968 , 7.8163195, 8.073858 ,
       6.992436 , 7.350293 , 7.9057307, 7.3783717, 7.1461086, 7.8354883,
   

### Create a Preprocessing layer

Let us now define a preprocessing layer that accepts a sentence as an input, removes punctuations and line-brk tags form it, then converts it's words into indices and finally output an embedding vector corresponding to the sentence.

In [148]:
class Preprocessing_layer(keras.layers.Layer): 
    def __init__(self, vocab,  num_oov_buckets, embedding_len, **kwargs): 
        super().__init__(**kwargs)
        self.vocab = list(vocab)
        self.vocab_len  = tf.size(self.vocab, out_type = tf.int64) 
        self.num_oov_buckets = num_oov_buckets
        self.embedding_len = embedding_len
        self.indices = tf.range(self.vocab_len, dtype = tf.int64)
        self.lookup_initializer = tf.lookup.KeyValueTensorInitializer(self.vocab, 
                                                                      self.indices)
        self.lookup_table = tf.lookup.StaticVocabularyTable(initializer = self.lookup_initializer, 
                                                            num_oov_buckets = self.num_oov_buckets)
         
    # build the embedding matrix 
    def build(self, batch_input_shape):
        self.embedding_mat = self.add_weight(name = 'embedding matrix', 
                                             shape = [self.vocab_len + self.num_oov_buckets, self.embedding_len],
                                             initializer = tf.random_uniform_initializer)
        
        super().build(batch_input_shape)
        
    
    # function to convert a sentence into a list of indices for its words
    def text_to_integers(self, X):
        X_words = tf.strings.split(X)
        # X_words will be a ragged tensor
        # on the otherhand, self.lookup_table.lookup() expects either a Sparse tensor or a dense tensor
        # for example see the documentation: https://www.tensorflow.org/api_docs/python/tf/lookup/StaticVocabularyTable
        # in order to use it ragged tensors, we have to use tf.ragged.map_flay_values()
        # https://www.tensorflow.org/api_docs/python/tf/ragged/map_flat_values
        X_int = tf.ragged.map_flat_values(self.lookup_table.lookup, X_words)
        return X_int
    
    # function to convert a list of indices into an embedding vector
    def integers_to_embeddings(self, X):
        sqrt_num_words = tf.linalg.diag(
            tf.math.sqrt(tf.cast(X.nested_row_lengths(), tf.float32))[0])
        X_ems = tf.nn.embedding_lookup(self.embedding_mat, X)
        X_vec = tf.math.reduce_mean(X_ems, axis = 1) # note that X_ems is a ragged tensor whose 
                                                     # 0-th dim. corresponds to the batch dim. 
                                                     # therefore axis = 1 and NOT 0 when taking mean
                                                     
        
        print(X_vec)
        
        return sqrt_num_words@X_vec
        
    # define the call method
    def call(self, X):
        
        # recall X contains strings with all the punctuations etc removed
        # Imp: The shape of X is (batch_size, ...) i.e. the 0-th axis of X corresponds to batch dim. 
        # We first convert all its words into their indices
        X_int = self.text_to_integers(X)
        # now convert this a mean embedding vector
        X_vec = self.integers_to_embeddings(X_int)
        
        return X_vec
    
    
    # compute output shape
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.embedding_len])
    
    
    # add the layers hyperparameters to the configuration file
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'embedding_len': self.embedding_len, 
                'num_oov_buckets': self.num_oov_buckets}
    

### Labeling the data

Note that the positive and negative reviews in each dataset have been stored in seperate folders and as such they have no explicit labels. We will therefore have to create our own labels as was done in [this](https://www.tensorflow.org/tutorials/load_data/text) official tensorflow tutorial. 

In [48]:
# function to return apply a label
def labeler(review, label):
    return review, tf.cast(label, tf.int64)

In [49]:
train_pos_paths = tf.data.Dataset.list_files(train_pos_path)
train_pos_reviews = tf.data.TextLineDataset(train_pos_paths).map(lambda x : labeler(x, 1))

In [50]:
for item in train_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"I think the comments regarding the show being cheesy are a bit too exaggerated. When a person comes to watch a TV show, what does he look out for? It is to enjoy that he watches a show, unless he/she is a critic or a person who analyzes story. But most of us are not so and watch the shows to relax and enjoy. FULL HOUSE is an ideal show to watch after having a heavy day in the office/school. It makes you laugh and it is not just humor.<br /><br />Yes, the Tanner family is a perfect family, a perfectly hypothetical family. If any such family existed in real world, it would be a role model for us to follow. But this is a TV show, and not a real family, and there is nothing wrong in depicting a hypothetical family on television. The very fact that the show could run so long shows us that people enjoyed watching it, whatever be the comments later on.<br /><br />Another good point about the show is that any person of any age would not only enjoy w

In [51]:
train_neg_paths = tf.data.Dataset.list_files(train_neg_path)
train_neg_reviews = tf.data.TextLineDataset(train_neg_paths).map(lambda x: labeler(x, 0))

In [52]:
for item in train_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"Just like last years event WWE New Years Revolution 2006 was headlined by an Elimination Chamber match. The difference between last years and this years match however was the entertainment value. In reality only three people stood a chance of walking out of the Pepsi Arena in Albany, New York with the WWE Championship. Those men were current champion John Cena, Kurt Angle and Shawn Michaels. There was no way Vinnie Mac would put the belt on any of the rookies; Carlito or Chris Masters. And Kane? Kane last held the WWE Championship in June 1998, and that was only for one night. It was obvious he wasn't going to be the one either. Last years match was a thrilling affair with six of the best WWE had to offer. 2006 was a predictable and disappointing affair but still the match of the night by far.<br /><br />The only surprise of the evening came after the bell had run on the main event. Out strolled Vince McMahon himself and demanded they lift t

### Creating training batches

In [53]:
shuffle_buffer_size = 20000
train_dataset = train_pos_reviews.concatenate(train_neg_reviews).shuffle(buffer_size = shuffle_buffer_size)

In [54]:
count = 0
for item in train_dataset:
    count+=1
print(count)    

25000


We now wish to apply punc_filter_and_to_lower4() to the reviews in our training dataset. For this we will use the [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) method of tf.data.Datasets. At this point it is important to note that (as mentioned in the documentation for [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map)):

    Note that irrespective of the context in which map_func is defined (eager vs. graph), tf.data traces the function and executes it as a graph. To use Python code inside of the function you have two options:

     1) Rely on AutoGraph to convert Python code into an equivalent graph computation. The downside of this approach is that AutoGraph can convert some but not all Python code.

     2) Use tf.py_function, which allows you to write arbitrary Python code but will generally result in worse performance than 1)

This point is of concern to us because the [tokenizer](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) we used in punc_filter_and_to_lower4() only accepts string input and NOT tensors. This string was extracted from the tensor using its .numpy() method which is only operable in eager mode. Thus when we pass punc_filter_and_to_lower4() to map(), it throws an error, complaining:

     'Tensor' object has no attribute 'numpy'


Thus we have to wrap punc_filter_and_to_lower4() with tf.py_function before passing it to map


In [55]:
# function to wrap punc_filter_and_to_lower4 with tf.py_function
# we will also use tf.ensure_shape(), or else tensorflow is 
# unable to statically determine it, leading to error being thrown during training
def string_transform(X):
    x = tf.py_function(punc_filter_and_to_lower4, [X], Tout = tf.string)
    x = tf.ensure_shape(x, ())
    
    return x

batch_size = 50
prefetch = 2
train_batch = train_dataset.map(lambda X, y: 
                                    (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

Note that we could have alternatively, preprocessed the data using punc_filter_and_to_lower2() which is based on tensorflow functions and then used it. The price we would have to pay is to rebuild the vocabulary accordingly. 

In [56]:
ite = next(iter(train_batch))
print('shape of train-batch: {}'.format(ite[0].shape))
print(ite[0][0])

shape of train-batch: (50,)
tf.Tensor(b'the true story of the friendship that shook south africa and awakened the world richard attenborough who directed a bridge too far and gandhi wanted to bring the story of steve biko to life and the journey and trouble that journalist donald woods went through in order to get his story told the films uses wood s two books for it s information and basis biko and asking for trouble the film takes place in the late 1970 s in south africa south africa is in the grip of the terrible apartheid which keeps the blacks separated from the whites and classifies the whites as the superior race the blacks are forced to live in shantytowns on the outskirts of the cities and towns and they come under frequent harassment by the police and the army we are shown a dawn raid on a shantytown as bulldozers and armed police force their way through the camp beating and even killing the inhabitants then we are introduced to donald woods kevin kline who is the editor of a

In [57]:
ite[1]

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1])>

### Create Validation dataset 

As instructed in the excercise, we will create a validation dataset by splitting the test set into half

In [58]:
test_path = os.path.join(path, 'test')
print(test_path)
test_pos_path = os.path.join(test_path, 'pos', '*.txt')
print('path to positive test reviews: \n{}'.format(test_pos_path))
test_neg_path = os.path.join(test_path, 'neg', '*.txt')
print('path to negative test reviews: \n{}'.format(test_neg_path))

/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test
path to positive test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/pos/*.txt
path to negative test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/neg/*.txt


In [59]:
test_pos_files = glob.glob(test_pos_path)
len(test_pos_files)

12500

In [60]:
valid_size = 12500
valid_pos_files = random.sample(test_pos_files, int(valid_size/2))
len(valid_pos_files)

6250

In [61]:
# We can remove the valid_pos_files from the set of test_pos_files through the
# simple trick explained in the following post:
# https://stackoverflow.com/questions/6486450/python-compute-list-difference/6486467
test_pos_files = list(set(test_pos_files) - set(valid_pos_files))
len(test_pos_files)

6250

In [62]:
test_neg_files = glob.glob(test_neg_path)
print('initial test_neg_files len:{}'.format(len(test_neg_files)))
valid_neg_files = random.sample(test_neg_files, int(valid_size/2))
print('valid_neg_files len: {}'.format(len(valid_neg_files)))
test_neg_files = list(set(test_neg_files) - set(valid_neg_files))
print('test_neg_files len: {}'.format(len(test_neg_files)))

initial test_neg_files len:12500
valid_neg_files len: 6250
test_neg_files len: 6250


In [63]:
valid_pos_reviews = tf.data.TextLineDataset(valid_pos_files).map(lambda x: labeler(x, 1))
valid_neg_reviews = tf.data.TextLineDataset(valid_neg_files).map(lambda x: labeler(x, 0))

In [64]:
for item in valid_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'A beautifully photographed and paced short film. It evocatively captures the feeling of this family and much of the country during the period just prior to and after Pearl Harbor.<br /><br />I appreciated the visual look of the film -- naturalistic and simultaneously poetic. Great work by a great D.P., David Boyd.<br /><br />Though a family film, the story never becomes maudlin or saccharin. We understand and believe the motivation that propels the young boy on his odyssey. I understand the love of the younger brother for his older brother and do not question why he sets out to do what he does. I understand that he is driven by a deep desire to be with his brother in this time of crisis. The kid is tough, and the performance by Jonathan Furr is superb as is the veteran performance by Ron Perlman.'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"Symbologist Robert Langdon (Hanks) is called to Rome

In [65]:
for item in valid_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"When I first saw this I thought bits of it were fairly scary. But the horror is a little undermined by the fact that the dozy teenagers are so irritating that you don't really care what happens to them. There are some genuinely good moments such as Angela's speech about demons, and the briefly-seen demon itself is effective. But this really doesn't stand up to repeated viewing or close analysis. It's all pretty tacky and cheap-looking and often downright silly, pornography and unfunny wisecracks all too often replacing horror. The music is excellent, however, and the epilogue is certainly amusing, but this film doesnt have much to recommend it, just comes across as another Jason-type stalk-and-slash fare.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"Well well, at last a view of this underrated flick. But you can't find a good copy of it, terrible copy full with green drops, the editing isn't 

In [66]:
test_pos_reviews = tf.data.TextLineDataset(test_pos_files).map(lambda x: labeler(x, 1))
test_neg_reviews = tf.data.TextLineDataset(test_neg_files).map(lambda x: labeler(x, 0))

In [67]:
for item in test_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"King of the Underworld features an early role for Humphery Bogart in one of his many gangster roles.<br /><br />He plays Joe Gurney who uses a female doctor to treat his men and pays her for it. He follows her when she goes to live with her Auntie after one of Gurney's men kills her doctor husband who also worked for him. Gurney kidnaps an author on his way to find the female doctor and gets him to write his life story and he then plans to kill him. He finally meets up with the doctor and after she gives Gurney and his men a substance that makes them temporarily blind, she and the author, who have now fallen in love manage to escape just as police arrive...<br /><br />Joing the excellent Bogie in the cast are Kay Francis, James Stephenson and John Eldredge.<br /><br />Watching King of the Underworld is a good way to spend just over an hour one evening.<br /><br />Rating: 3 stars out of 5.">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
 
(<t

In [68]:
for item in test_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'The price of a dream - and some dreams can be "too" expensive.<br /><br />Only having viewed the English-translated version, it is perhaps the reason for a low rating from this viewer.<br /><br />It made the overall film poorer than the story material hinted at...<br /><br />...and other comments seem to suggest the subtitled version would be better.<br /><br />But some plot elements remained unexplained, leaving an unfinished feel.<br /><br />It also leaves the thought "is there a series to follow?".<br /><br />A pity there was no more (at this stage at least, anyway).'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"The first review I saw of this on IMDB says that Vince Vaughn is a much better actor than Anthony Perkins was in this role. Makes me wonder if he saw the original. It's tough to review Psycho if you don't have the perspective of how revolutionary the movie was in 1960. You have a he

In [69]:
buffer_size = 10000
valid_dataset = valid_pos_reviews.concatenate(valid_neg_reviews).shuffle(buffer_size = buffer_size)

In [70]:
valid_batch = valid_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

In [71]:
test_dataset = test_pos_reviews.concatenate(test_neg_reviews).shuffle(buffer_size)

In [72]:
test_batch = test_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

### A simple model to test the preprocessing layer defined earlier

In [85]:
num_oov_buckets = 50
embedding_len = 100

In [92]:
item = next(iter(train_batch.take(1)))

In [149]:
model = keras.Sequential()
model.add(keras.layers.Input(shape = [], dtype = tf.string))
model.add(Preprocessing_layer(vocab = vocab, 
                              num_oov_buckets = num_oov_buckets, 
                              embedding_len = embedding_len) )

Tensor("preprocessing_layer_20/RaggedReduceMean/truediv:0", shape=(None, 100), dtype=float32)


In [150]:
pre = model(item[0])

tf.Tensor(
[[-6.2578367e-03 -1.8596064e-03  8.6801304e-03 ... -3.5115203e-03
  -5.4248427e-03  4.7876956e-03]
 [-1.0758750e-03  6.4234257e-05 -3.9096415e-04 ... -2.1506080e-03
  -2.8772862e-03  2.5930160e-03]
 [-5.0878073e-03 -5.1251025e-04  1.1080215e-03 ... -1.6537699e-03
  -8.2211923e-03  3.1295759e-03]
 ...
 [-2.3760927e-04 -2.2121628e-03  1.4615195e-03 ... -3.4966464e-03
  -1.6592853e-03  5.7384861e-03]
 [-1.2913959e-03 -3.0809417e-03 -2.1347328e-04 ... -2.9889084e-03
  -3.0697582e-03  5.5458606e-03]
 [-3.6643252e-03 -2.3131960e-03  6.2308838e-03 ... -1.9693079e-03
  -6.3814642e-04 -5.1055168e-04]], shape=(50, 100), dtype=float32)
